In [1]:
import sys
import time
import json
import os
#sys.path.insert(0, 'src/data')
#sys.path.insert(0, 'src/dgl_graphsage')

import recommend as r
from train_updated import train
from api.spotifyAPI import SpotifyAPI
from utils import load_graph
from utils import load_features
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from dgl import load_graphs
import matplotlib.pyplot as plt
import networkx as nx
import dgl

import torch
import numpy as np

Using backend: pytorch


# Loading Data

In [2]:
%%time
# Loading Graph ~ 3min
feat_dir = "../../data/a13group1/features/merged_features.csv"
double_edge_dir = "../../data/a13group1/double_edges_170k.bin"
feat_data, uri_map = load_features(feat_dir)
dgl_G = nx.read_gpickle("../../data/a13group1/graph_170k.gpickle")
dgl_G, weights = load_graph(dgl_G, uri_map)

CPU times: user 1min 58s, sys: 28.4 s, total: 2min 27s
Wall time: 2min 25s


In [3]:
#dgl_G_cuda = dgl_G.to('cuda')
#weights = weights.to('cuda')
#feat_data = feat_data.to('cuda')

# Training Model (optional)

In [4]:
# %%time
# # Training the Model. GPU ~ 00:00:40. CPU ~ 00:53:00.
# with open('../../config/model-params.json') as fh:
#             model_cfg = json.load(fh)
# model, pred, losses = train(dgl_G, weights.to('cpu'), feat_data, cuda=False, feat_dim=14, emb_dim=10, test_data=False)

# # Put everything on CPU
# model = model.to('cpu')
# pred = pred.to('cpu')

# torch.save(model, '170k_model.pt')
# torch.save(pred, '170k_pred.pt')

# Loading Pre-Existing Model

In [5]:
# Load Pre-Existing Model
model = torch.load('170k_1epoch_model.pt')
pred = torch.load('170k_1epoch_pred.pt')
model.eval()
pred.eval()

MLPPredictor(
  (W1): Linear(in_features=20, out_features=10, bias=True)
  (W2): Linear(in_features=10, out_features=1, bias=True)
)

# Create Predictions

In [6]:
dgl_G.edata[dgl.EID] = torch.tensor(np.arange(dgl_G.number_of_edges()))

In [7]:
# Create Embeddings
z = model(dgl_G, feat_data, weights)

# Create Predictions
# In the form of the strength of the connection between source, destination from dgl_G.edges()
preds = pred(dgl_G, z)

In [8]:
# Create Nearest Neighbors
neigh = NearestNeighbors(n_neighbors=25, radius=0.4)
neigh.fit(feat_data)

NearestNeighbors(n_neighbors=25, radius=0.4)

In [9]:
# Get a random playlist
item = r.get_random_playlist()

/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.0-999.json


In [10]:
item

{'name': 'Best',
 'collaborative': 'false',
 'pid': 739,
 'modified_at': 1435190400,
 'num_tracks': 42,
 'num_albums': 39,
 'num_followers': 3,
 'tracks': [{'pos': 0,
   'artist_name': 'Sub Focus',
   'track_uri': 'spotify:track:3VLi4kIsnLMHd2bB5mPlfi',
   'artist_uri': 'spotify:artist:0QaSiI5TLA4N7mcsdxShDO',
   'track_name': 'Endorphins',
   'album_uri': 'spotify:album:6qeDiApbWmoxPIDHnM2r98',
   'duration_ms': 246889,
   'album_name': 'Endorphins'},
  {'pos': 1,
   'artist_name': 'Sub Focus',
   'track_uri': 'spotify:track:0iJs2bd00pmNa4RNFn18I8',
   'artist_uri': 'spotify:artist:0QaSiI5TLA4N7mcsdxShDO',
   'track_name': 'Turn It Around',
   'album_uri': 'spotify:album:5VDg2zP4jsc8Ufj1dPEA3d',
   'duration_ms': 194560,
   'album_name': 'Torus'},
  {'pos': 2,
   'artist_name': 'Bassnectar',
   'track_uri': 'spotify:track:3x94KfC5QAsqFKjUGQBH4k',
   'artist_uri': 'spotify:artist:1JPy5PsJtkhftfdr6saN2i',
   'track_name': 'You & Me (feat. W. Darling)',
   'album_uri': 'spotify:album:7gY

In [11]:
# Get playlist information
item, original_tracks, seeds = r.get_playlist_info(item)
original_tracks

Playlist ID: 739
Playlist Length: 42


['Endorphins---Sub Focus',
 'Turn It Around---Sub Focus',
 'You & Me (feat. W. Darling)---Bassnectar',
 'Collapse---Zeds Dead',
 'Sleepless - Oliver Nelson Remix---CAZZETTE',
 'High You Are---What So Not',
 'Don’t Leave---Seven Lions',
 'Derezzed - Remixed by Avicii---Daft Punk',
 'This Is How You Know---Coyote Kisses',
 'Rain (MitiS Remix)---Astronaut',
 'Always - Classixx Remix---Panama',
 'Classic (feat. POWERS)---The Knocks',
 'Treat Me Right---Keys N Krates',
 'Tilt Mode (feat. Gibbz)---Exmag',
 'Say My Name (feat. Zyra) - RAC Mix---ODESZA',
 'Understand Why - Original Mix---Keys N Krates',
 'Ridin Solo - Njomza Remix---Hippie Sabotage',
 'Let Go - Sir Sly Remix---RAC',
 'Cheap Sunglasses - Le Youth Remix---RAC',
 'A Stranger Love (Salva Remix)---Classixx',
 'Sway - Chainsmokers Remix---Anna Of The North',
 'Out Of My Head - Tobtok Remix---Zak Waters',
 'All I Want - Manila Killa Remix---Dawn Golden',
 'Whoops (feat. Pennybirdrabbit)---12th Planet',
 'Tidal Wave---Sub Focus',
 'Al

In [12]:
# Get recommendations
uri_recs = r.recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
uri_recs

['7CMSR4t19k6UAL2sHXUQ9X',
 '29oqqEp0tTjMbSnExkuNFC',
 '1wHZx0LgzFHyeIZkUydNXq',
 '6p8NuHm8uCGnn2Dtbtf7zE',
 '78SAVldsO3Xzm2R2Ko6E66',
 '6p8NuHm8uCGnn2Dtbtf7zE',
 '7BKLCZ1jbUBVqRi2FVlTVw',
 '4X5f3vT8MRuXF68pfjNte5',
 '5dNfHmqgr128gMY2tc5CeJ',
 '6OTUZuPrTU8nVYjJM3LYw8',
 '3JtmRIkVK95rmf4kWKA3ON',
 '03fT3OHB9KyMtGMt2zwqCT',
 '4X5f3vT8MRuXF68pfjNte5',
 '7yyRTcZmCiyzzJlNzGC9Ol',
 '2EEeOnHehOozLq4aS0n6SL',
 '4Kzrd46VFJIYhY3CinY7bQ',
 '6p8NuHm8uCGnn2Dtbtf7zE',
 '7KXjTSCq5nL1LoYtL7XAwS',
 '7BKLCZ1jbUBVqRi2FVlTVw',
 '7BKLCZ1jbUBVqRi2FVlTVw',
 '1zWZvrk13cL8Sl3VLeG57F',
 '6q5dZkmPJKaW0GXKHY9uoK',
 '6p8NuHm8uCGnn2Dtbtf7zE',
 '66hayvUbTotekKU3H4ta1f',
 '7KXjTSCq5nL1LoYtL7XAwS',
 '6DNtNfH8hXkqOX1sjqmI7p',
 '4Km5HrUvYTaSUfiSGPJeQR',
 '7BKLCZ1jbUBVqRi2FVlTVw',
 '7yyRTcZmCiyzzJlNzGC9Ol',
 '1zWZvrk13cL8Sl3VLeG57F',
 '4w7yrP4RAeeyhfG9nJqQvS',
 '743H2NCU4LvZ0gh014GUxK',
 '5NQbUaeTEOGdD6hHcre0dZ',
 '4O1CExxinEpKZi3861NlTK',
 '2tux1s5FVabQQKpz7GEXbI',
 '7yyRTcZmCiyzzJlNzGC9Ol',
 '787fe3dyhuG04cYV3gUE39',
 

# Decode Results with Spotify API

In [13]:
# Translate to Song Names
client_id = 'ad2536ed7a914d66b89b80fb3a500787'
client_secret = '8c5f45fb008d4bc5bf909ec46d076b65'

spotify = SpotifyAPI(client_id, client_secret)

In [14]:
%%time
rec_track_names = r.get_rec_names(uri_recs, spotify, 2)

CPU times: user 45.8 ms, sys: 5.63 ms, total: 51.5 ms
Wall time: 2.38 s


In [15]:
original_tracks

['Endorphins---Sub Focus',
 'Turn It Around---Sub Focus',
 'You & Me (feat. W. Darling)---Bassnectar',
 'Collapse---Zeds Dead',
 'Sleepless - Oliver Nelson Remix---CAZZETTE',
 'High You Are---What So Not',
 'Don’t Leave---Seven Lions',
 'Derezzed - Remixed by Avicii---Daft Punk',
 'This Is How You Know---Coyote Kisses',
 'Rain (MitiS Remix)---Astronaut',
 'Always - Classixx Remix---Panama',
 'Classic (feat. POWERS)---The Knocks',
 'Treat Me Right---Keys N Krates',
 'Tilt Mode (feat. Gibbz)---Exmag',
 'Say My Name (feat. Zyra) - RAC Mix---ODESZA',
 'Understand Why - Original Mix---Keys N Krates',
 'Ridin Solo - Njomza Remix---Hippie Sabotage',
 'Let Go - Sir Sly Remix---RAC',
 'Cheap Sunglasses - Le Youth Remix---RAC',
 'A Stranger Love (Salva Remix)---Classixx',
 'Sway - Chainsmokers Remix---Anna Of The North',
 'Out Of My Head - Tobtok Remix---Zak Waters',
 'All I Want - Manila Killa Remix---Dawn Golden',
 'Whoops (feat. Pennybirdrabbit)---12th Planet',
 'Tidal Wave---Sub Focus',
 'Al

In [16]:
rec_track_names.sort(reverse=True, key=lambda x: x[1])

In [17]:
rec_track_names

[('Slippery (feat. Gucci Mane)---Migos', 5),
 ('Closer---The Chainsmokers', 4),
 ('Broccoli (feat. Lil Yachty)---Shelley FKA DRAM', 3),
 ('T-Shirt---Thomas Rhett', 2),
 ("F**kin' Problems (feat. Drake, 2 Chainz & Kendrick Lamar)---A$AP Rocky", 2),
 ('HUMBLE.---Kendrick Lamar', 2),
 ('Play It Again---Luke Bryan', 1),
 ('One Mississippi---WHY?', 1),
 ('Antidote---Travis Scott', 1),
 ('Secrets Inside---Swain', 1),
 ('iSpy (feat. Lil Yachty)---KYLE', 1),
 ('Lost, Scared & Tired---Jared & The Mill', 1),
 ('Stay A Little Longer---Brothers Osborne', 1),
 ('Bad and Boujee (feat. Lil Uzi Vert)---Migos', 1),
 ('Riot---Designer Drugs', 1),
 ('Get Me Some Of That---Thomas Rhett', 1),
 ("I Don't Fuck With You---Big Sean", 1),
 ('Alone - Amtrac Remix---Amtrac', 1),
 ('679 (feat. Remy Boyz)---Fetty Wap', 1),
 ('Ignition - Remix---R. Kelly', 1),
 ('Where Are Ü Now (with Justin Bieber)---Jack Ü', 1),
 ('Cold Water (feat. Justin Bieber & MØ)---Major Lazer', 1),
 ('Roses (feat. ROZES)---The Chainsmokers'

Note: We kept repeats because a song that is recommended as the best more than once throughout the playlist should be ranked higher

# Making Batch Recommendations

In [22]:
%%time
all_accs = []
for each in range(50):
    item = r.get_random_playlist()
    item, original_tracks, seeds = r.get_playlist_info(item)
    uri_recs = r.recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
    
    checks = []
    for i in range(len(uri_recs)):
        well = uri_map[uri_recs[i]] in dgl_G.out_edges(uri_map[seeds[i]])[1]
        checks.append(well)
        
    acc = np.count_nonzero(checks) / len(checks)
    all_accs.append({'pid': item['pid'], 'seeds': seeds, 'recs':uri_recs, 'accuracy': acc})
    
with open("../analysis/batch_recommendations.json", "w") as final:
    json.dump(all_accs, final)

/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.8000-8999.json
Playlist ID: 8710
Playlist Length: 34
CPU times: user 2.36 s, sys: 144 ms, total: 2.5 s
Wall time: 1.72 s
